# 4. Assignment

## Portfolio Management
### MLP Training

#### January 2025

Questions?
* Contact: Mark Hendricks
* hendricks@uchicago.edu

***

# 1. Carry in Equities

### Data
You will need the files in the github repo, 
* `data/spx_carry_weekly.xlsx`
* `data/spx_returns_weekly.xlsx`

### 1. Data Processing
* Take the intersection of dates of the two data sets, so you have one unified set of dates.
* Report the maximum and minimum dividend yields from the entire panel of data. To which stocks did they belong?

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

SCALE = 52
dvd_yld_df = pd.read_excel('../data/spx_carry_weekly.xlsx', 's&p500 dvd yield').set_index('date')
ret_df = pd.read_excel('../data/spx_returns_weekly.xlsx', 's&p500 rets').set_index('date')

In [12]:
display(dvd_yld_df.head())
display(ret_df.head())

,A,AAPL,ABBV,ABT,ACN,ADI,ADM,ADP,AEE,AEP,...,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.009862,0.017196,0.029747,0.021381,0.022963,0.026647,0.018454,0.023532,0.035660,0.034669,...,0.008436,0.026936,0.032124,0.040847,0.033223,0.029732,0.013445,0.031524,0.008057,0.007666
2015-01-09,0.009855,0.016784,0.029796,0.021239,0.022722,0.027047,0.019664,0.023179,0.036444,0.034393,...,0.008496,0.026816,0.031763,0.040255,0.033167,0.029967,0.014476,0.031048,0.007674,0.007503
2015-01-16,0.010458,0.017738,0.030369,0.021578,0.022950,0.027566,0.020134,0.023176,0.035239,0.033349,...,0.008484,0.027322,0.031668,0.040819,0.032415,0.030290,0.014853,0.031489,0.007753,0.007508
2015-01-23,0.010307,0.016640,0.031395,0.021888,0.022865,0.027151,0.020054,0.022568,0.035398,0.032858,...,0.008679,0.027759,0.032186,0.041093,0.031780,0.030366,0.014558,0.030776,0.007754,0.007525
2015-01-30,0.010590,0.016046,0.032477,0.021448,0.024277,0.028404,0.020588,0.023749,0.036219,0.033753,...,0.008923,0.027714,0.032357,0.040554,0.031974,0.031572,0.015015,0.031555,0.008092,0.007770


,A,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-0.038352,0.001776,0.041190,0.003094,-0.028354,0.010896,0.015026,-0.003202,-0.083744,-0.002688,...,-0.035071,-0.074553,0.010911,0.008844,0.038694,-0.040441,-0.008856,-0.004628,-0.045570,-0.054490
2016-01-22,0.023443,0.044184,0.025986,-0.012578,-0.017597,0.029919,0.005159,0.031313,0.054585,0.040940,...,-0.005154,-0.015493,0.013707,0.003561,-0.013019,0.031835,0.028171,-0.019094,-0.000884,0.002658
2016-01-29,-0.008688,-0.040268,-0.066803,-0.054461,0.034298,0.032277,-0.005578,0.048268,0.063800,0.024411,...,0.022510,0.007475,0.138075,0.043407,0.016717,0.056423,0.049144,0.001109,0.069027,-0.008065
2016-02-05,-0.042762,-0.028849,-0.032423,-0.011625,-0.023833,-0.062252,-0.111410,-0.075010,-0.036211,-0.021423,...,-0.003931,-0.063255,-0.080634,0.034274,0.028646,0.019748,-0.036065,-0.042816,-0.050000,-0.049711
2016-02-12,0.004993,-0.000327,-0.010165,-0.007484,0.037305,-0.043144,-0.031944,-0.003612,-0.038858,0.005288,...,-0.054202,-0.057943,0.116782,-0.017960,0.021046,0.010367,-0.034689,-0.033997,0.033635,-0.011732


In [13]:
dvd_yld_df, ret_df = dvd_yld_df.align(ret_df, join="inner", axis=0)

### 2. Static Carry
* Starting at the beginning of the sample, build a portfolio where you go (equally) long the top 100 stocks as ranked by dividend yield, and you short (equally) the smallest 100 stocks as ranked by dividend yield.
* Using those weights, calculate the return of your portfolio over time. 

Report the following annualized stats
* mean
* volatility
* Sharpe

In [4]:
# get the first row
first_row =  dvd_yld_df.iloc[0].sort_values()

# pick losers and winners
losers = first_row[0:100].index
winners = first_row[-100:].index
remaining = first_row.loc[~first_row.index.isin(losers) & ~first_row.index.isin(winners)].index

# replicate positions
positions = first_row.copy()
positions[losers] = 0.01
positions[winners] = -0.01
positions[remaining] = 0.0

positions.name = "Static Positions"
positions.head()


COO     0.01
CI      0.01
C       0.01
GPN     0.01
FICO    0.01
Name: Static Positions, dtype: float64

In [17]:
filtered_ret_df, filtered_positions = ret_df.align(positions, join="inner", axis=1)

filtered_ret_df.shape, filtered_positions.shape

((468, 392), (392,))

# Statistics on portfolio

In [19]:
port_ret = filtered_ret_df.dot(filtered_positions)
port_ret.name = 'Return'

In [20]:
port_mean_ret = port_ret.mean() * SCALE
port_mean_ret

np.float64(0.05053491764443579)

In [21]:
port_vol = port_ret.std() * np.sqrt(SCALE)
port_vol

np.float64(0.06896267209389502)

In [24]:
port_sharpe_ratio = port_mean_ret / port_vol
port_sharpe_ratio

np.float64(0.732786536688004)

In [33]:
print(pd.DataFrame(data=filtered_positions[filtered_positions < 0]).to_string())

       Static Positions
AEE               -0.01
APO               -0.01
BAX               -0.01
BKNG              -0.01
BX                -0.01
CAG               -0.01
CCI               -0.01
CNP               -0.01
COP               -0.01
CRM               -0.01
CTSH              -0.01
CVX               -0.01
DD                -0.01
DG                -0.01
DLR               -0.01
DOC               -0.01
DRI               -0.01
DTE               -0.01
DUK               -0.01
EA                -0.01
EBAY              -0.01
ED                -0.01
EQIX              -0.01
ETR               -0.01
EXC               -0.01
FANG              -0.01
FCX               -0.01
FE                -0.01
GE                -0.01
GILD              -0.01
GOOG              -0.01
GOOGL             -0.01
GRMN              -0.01
HCA               -0.01
HLT               -0.01
IRM               -0.01
J                 -0.01
KIM               -0.01
KKR               -0.01
KMI               -0.01
LH              

### 3. Challenges

Mention two challenges of this strategy.

We are shorting Tech stocks that perform really well:
- NVIDIA
- GOOG/GOOGL
- META

We go long on energy and utilities. This is terrible considering the past years were we have explosive growth in the tech sector. We need someway to hedge out the tech factor in this strategy.

### 4. Dynamic Carry

Make the carry strategy of `2` dynamic by recalculating the weights every period.

Report the performance statistics.

In [35]:
sorted_dvd_df = dvd_yld_df.apply(lambda row: sorted(row), axis=1, result_type="expand")

,0,1,2,3,4,5,6,7,8,9,...,406,407,408,409,410,411,412,413,414,415
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,0.006817,0.007517,0.007540,0.011967,0.012310,0.012396,0.013105,0.013831,0.013939,0.014670,...,0.074840,0.213615,NaN,NaN,NaN,NaN,NaN,0.053392,NaN,0.159006
2016-01-22,0.006671,0.007563,0.007674,0.011499,0.011585,0.012112,0.012753,0.013887,0.014135,0.014749,...,NaN,NaN,NaN,NaN,0.048708,0.052613,NaN,0.048780,0.072390,0.129686
2016-01-29,0.006605,0.007651,0.007781,0.011297,0.012218,0.012568,0.013163,0.013663,0.013744,0.013900,...,NaN,NaN,NaN,NaN,0.052784,NaN,0.048419,0.055288,0.070544,0.132642
2016-02-05,0.007942,0.007973,0.008530,0.011302,0.012270,0.012764,0.012921,0.013711,0.014106,0.014643,...,NaN,NaN,NaN,NaN,0.051847,NaN,0.051688,0.054499,0.068975,0.149533
2016-02-12,0.007890,0.007979,0.008348,0.011304,0.012565,0.012700,0.012793,0.014018,0.014467,0.014587,...,NaN,NaN,NaN,NaN,0.060293,NaN,0.054867,0.063235,0.069751,0.190052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-29,0.004214,0.005762,0.006728,0.006896,0.007190,0.007686,0.009085,0.009158,0.009190,0.010314,...,0.043624,0.045329,0.045752,0.047927,0.048040,0.053051,0.054621,0.061119,0.110865,NaN
2024-12-06,0.004118,0.005847,0.006603,0.007056,0.007143,0.008089,0.008894,0.009211,0.009247,0.010402,...,0.045113,0.045692,0.045990,0.046482,0.051907,0.054660,0.056293,0.063960,0.116822,NaN
2024-12-13,0.004030,0.004516,0.004598,0.004690,0.004822,0.005759,0.005869,0.005898,0.005977,0.006695,...,0.046974,0.050637,0.055573,0.057246,0.058261,0.064097,0.067240,0.074290,0.096246,NaN


In [ ]:
# get the first row
first_row =  dvd_yld_df.iloc[0].sort_values()

# pick losers and winners
losers = first_row[0:100].index
winners = first_row[-100:].index
remaining = first_row.loc[~first_row.index.isin(losers) & ~first_row.index.isin(winners)].index

# replicate positions
positions = first_row.copy()
positions[losers] = 0.01
positions[winners] = -0.01
positions[remaining] = 0.0

positions.name = "Static Positions"
positions.head()

### 5. Market Exposure

Calculate a linear factor decomposition of your carry strategy (from part `2` or part `4`) against `SPY`.

Report the...
* alpha (annualized)
* beta
* r-squared

***